In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from typing import Optional
from functools import partial

import dspy
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

from sisyphus.heas.label import label_paras
from sisyphus.chain.paragraph import Paragraph, ParagraphExtend
from sisyphus.chain import Filter, Writer
from sisyphus.strategy.run_strategy import extract_main
from sisyphus.strategy.pydantic_models_general import Processing, Material, MaterialDescriptionBase
from sisyphus.strategy.utils import build_process_agent, build_property_agent, build_result_model_contextualized, get_paras_with_props, get_synthesis_paras
from sisyphus.heas.prompt import *
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.heas.synthesis import get_synthesis_prompt


lm = dspy.LM('openai/gpt-4.1')
dspy.configure(lm=lm)
chat_model = ChatOpenAI(model='gpt-4.1-mini')

class StrengthTestBase(BaseModel):
    """Tensile/Compressive test results"""
    ys: Optional[str] = Field(description="Yield strength with unit")
    uts: Optional[str] = Field(description="Ultimate tensile/compressive strength with unit")
    strain: Optional[str] = Field(description="Fracture strain with unit %")
    temperature: Optional[str] = Field(description="Test temperature with unit")
    strain_rate: Optional[str] = Field(description="Strain rate with unit")
    other_test_conditions: str = Field(description="Other test conditions, like in salt, hydrogen charging, etc.")

class PhaseInfo(BaseModel):
    """Phase information"""
    phases: list[str] = Field(description="List of phases present in the material")

strength_agent = build_property_agent(
    EXTRACT_PROPERTY_SYS_GENERIC_PROMPT,
    STRENGTH_PROMPT,
    build_result_model_contextualized('Strength', 'Extract tensile/compressive test information from provided text', StrengthTestBase, MaterialDescriptionBase),
    chat_model
)

phase_agent = build_property_agent(
    EXTRACT_PROPERTY_SYS_GENERIC_PROMPT,
    PHASE_PROMPT,
    build_result_model_contextualized('Phase', 'Extract phase information', PhaseInfo, MaterialDescriptionBase),
    chat_model
)

process_agent = build_process_agent(
    EXTRACT_PROCESS_SYS_GENERIC_PROMPT,
    PROCESS_PROMPT,
    Processing,
    chat_model
)

agents_d = {
    'strength': strength_agent,
    'phase': phase_agent
}

def reconstruct_p(paragraphs):
    p_str = ParagraphExtend.from_paragraphs(get_synthesis_paras(paragraphs) + get_paras_with_props(paragraphs, 'strength'), type='strength')
    p_phase = ParagraphExtend.from_paragraphs(get_synthesis_paras(paragraphs) + get_paras_with_props(paragraphs, 'phase'), type='phase')
    p_exp = ParagraphExtend.from_paragraphs(get_synthesis_paras(paragraphs) + get_paras_with_props(paragraphs, 'composition'), type='synthesis')
    return{
        "strength": p_str,
        "phase": p_phase,
        "synthesis": p_exp
    }



In [3]:
extract = partial(extract_main, reconstruct_paragraph=reconstruct_p, property_agents_d=agents_d, formatted_func=get_synthesis_prompt, synthesis_agent=process_agent, save_to='test_out.json')

db = get_plain_articledb('heas_1531')
getter = Filter(db)

chain = getter + label_paras + extract
chain.compose('10.1016&sol;j.jallcom.2021.161518.html')

e:\anaconda\envs\sisyphus_heas\lib\site-packages\pydantic\main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` with value `[{'induction melting': {'...m': 'air', 'rate': ''}}]` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
e:\anaconda\envs\sisyphus_heas\lib\site-packages\pydantic\main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` with value `[{'induction melting': {'...atmosphere': 'vacuum'}}]` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
e:\anaconda\envs\sisyphus_heas\lib\site-packages\pydantic\main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` with value `[{'induction melting': {'...atmosphere': 'vacuum'}}]` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [4]:
import json
s_ = """{"doi": "10.1016/j.jallcom.2021.161518", "results": [{"properties": [{"Strength": {"composition": "(CoCrFeNi)95Ti1Nb1Al3", "description": "annealed at 1273 K for 1 h", "refered": false, "ys": "190 MPa", "uts": "580 MPa", "strain": null, "temperature": null, "strain_rate": null, "other_test_conditions": ""}}, {"Phase": {"composition": "CoCrFeNiTiNbAl", "description": "annealed at 1273 K for 1 h", "refered": false, "phases": ["FCC"]}}], "synthesis": {"composition": "(CoCrFeNi)95Ti1Nb1Al3", "steps": [{"induction melting": {"vacuum": true, "atmosphere": "", "remelting times": ""}}, {"homogenized": {"temperature": "1473 K", "duration": "12 h", "atmosphere": ""}}, {"cold rolled": {"reduction": "", "temperature": ""}}, {"solution treated": {"vacuum": true, "temperature": "1273 K", "duration": "1 h", "atmosphere": "10^{-5} Pa vacuum"}}, {"cooling": {"medium": "air", "rate": ""}}]}}, {"properties": [{"Strength": {"composition": "(CoCrFeNi)95Ti1Nb1Al3", "description": "aged at 1023 K for 8 h", "refered": false, "ys": "450 MPa", "uts": "870 MPa", "strain": "40 %", "temperature": "1023 K", "strain_rate": null, "other_test_conditions": ""}}], "synthesis": {"composition": "(CoCrFeNi)95Ti1Nb1Al3", "steps": [{"induction melting": {"vacuum": true, "atmosphere": "", "remelting times": ""}}, {"homogenized": {"temperature": "1473 K", "duration": "12 h", "atmosphere": ""}}, {"cold rolled": {"reduction": "to 0.1 or 0.2 mm thickness", "temperature": "room temperature"}}, {"solution treated": {"vacuum": true, "temperature": "1273 K", "duration": "1 h", "atmosphere": "10^-5 Pa vacuum"}}, {"cooling": {"medium": "air", "rate": ""}}, {"aged": {"temperature": "1023 K", "duration": "8 h", "atmosphere": "vacuum"}}]}}, {"properties": [{"Phase": {"composition": "CoCrFeNiTiNbAl", "description": "annealed at 1273 K for 1 h and aged at 1023 K for 8 h", "refered": false, "phases": ["FCC", "L12 γ' precipitates"]}}], "synthesis": {"composition": "(CoCrFeNi)95Ti1Nb1Al3", "steps": [{"induction melting": {"vacuum": true, "atmosphere": "", "remelting times": ""}}, {"homogenized": {"temperature": "1473 K", "duration": "12 h", "atmosphere": ""}}, {"cold rolled": {"reduction": "to 0.1-0.2 mm thickness", "temperature": "room temperature"}}, {"solution treated": {"vacuum": true, "temperature": "1273 K", "duration": "1 h", "atmosphere": "10^-5 Pa vacuum"}}, {"aged": {"temperature": "1023 K", "duration": "8 h", "atmosphere": "vacuum"}}]}}, {"properties": [{"Strength": {"composition": "CoCrFeNi", "description": "annealed at 1273 K for 1 h", "refered": true, "ys": "184 MPa", "uts": "523 MPa", "strain": null, "temperature": null, "strain_rate": null, "other_test_conditions": ""}}], "synthesis": null}, {"properties": [{"Strength": {"composition": "CoCrFeMnNi", "description": "annealed at 1273 K for 1 h", "refered": true, "ys": "230 MPa", "uts": "760 MPa", "strain": null, "temperature": null, "strain_rate": null, "other_test_conditions": ""}}], "synthesis": null}]}
"""
with open('test_out.json', 'w') as f:

    f.write(json.dumps(json.loads(s_), ensure_ascii=False, indent=2))